<a href="https://colab.research.google.com/github/subodhkhanger/CarND-Behavioral-Cloning-P3/blob/master/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!wget -c https://d17h27t6h515a5.cloudfront.net/topher/2016/December/584f6edd_data/data.zip
!unzip -qq data.zip


--2019-01-01 03:58:01--  https://d17h27t6h515a5.cloudfront.net/topher/2016/December/584f6edd_data/data.zip
Resolving d17h27t6h515a5.cloudfront.net (d17h27t6h515a5.cloudfront.net)... 13.35.127.229, 13.35.127.40, 13.35.127.195, ...
Connecting to d17h27t6h515a5.cloudfront.net (d17h27t6h515a5.cloudfront.net)|13.35.127.229|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 333137665 (318M) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>] 317.70M  56.5MB/s    in 5.8s    

2019-01-01 03:58:08 (55.1 MB/s) - ‘data.zip’ saved [333137665/333137665]



In [0]:
!rm -r __MACOSX || true

In [1]:
import numpy as np
import math
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from collections import deque
from scipy.stats import norm
import random
%matplotlib inline


import tensorflow as tf

import cv2
import json




from pathlib import PurePosixPath
from collections import deque

from keras.utils import np_utils
from keras.layers import Conv2D, Flatten, Lambda
from keras.layers.pooling import MaxPooling2D
from keras.layers.core import Dropout, Dense, Activation
from keras.optimizers import SGD, Adam
from keras.models import Sequential
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping

Using TensorFlow backend.


In [0]:
log_path = './data'
column_names = ['center', 'left', 'right',
                'steering', 'throttle', 'brake', 'speed']
data_df = pd.read_csv(log_path+'/driving_log.csv',names=column_names)

In [0]:
data_df.tail()

,center,left,right,steering,throttle,brake,speed
8032,IMG/center_2016_12_01_13_46_38_745.jpg,IMG/left_2016_12_01_13_46_38_745.jpg,IMG/right_2016_12_01_13_46_38_745.jpg,0,0,0,1.402436
8033,IMG/center_2016_12_01_13_46_38_802.jpg,IMG/left_2016_12_01_13_46_38_802.jpg,IMG/right_2016_12_01_13_46_38_802.jpg,0,0,0,1.393976
8034,IMG/center_2016_12_01_13_46_38_846.jpg,IMG/left_2016_12_01_13_46_38_846.jpg,IMG/right_2016_12_01_13_46_38_846.jpg,0,0,0,1.388364
8035,IMG/center_2016_12_01_13_46_38_922.jpg,IMG/left_2016_12_01_13_46_38_922.jpg,IMG/right_2016_12_01_13_46_38_922.jpg,0,0,0,1.377208
8036,IMG/center_2016_12_01_13_46_38_947.jpg,IMG/left_2016_12_01_13_46_38_947.jpg,IMG/right_2016_12_01_13_46_38_947.jpg,0,0,0,1.374433


In [0]:
flags = tf.app.flags
FLAGS = flags.FLAGS

# command line flags




cameras = ['left', 'center', 'right']
camera_centre = ['center']
steering_adj = {'left': 0.25, 'center': 0., 'right': -.25}


#using open cv function to load image 
def loadImg(imgLoc,trainFile):
    filename = trainFile.strip()
    if filename.startswith('IMG'):
        filename = imgLoc+'/'+filename
    else:
        # load it relative to where log file is now, not whats in it
        filename = imgLoc+'/IMG/'+PurePosixPath(filename).name
    img = cv2.imread(filename)
   
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)



def brightness(image):
    hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    bv = .25 + np.random.uniform()
    hsv[::2] = hsv[::2]*bv

    return cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB)


def crop_img(img, crop_height=66, crop_width=200):
    height = img.shape[0]
    width = img.shape[1]
    y_start = 60
    x_start = int(width/2)-int(crop_width/2)

    return img[y_start:y_start+crop_height, x_start:x_start+crop_width]



def imagerotation(image, steering):
    rows, cols, _ = image.shape
    transRange = 100
    numPixels = 10
    valPixels = 0.4
    transX = transRange * np.random.uniform() - transRange/2
    steering = steering + transX/transRange * 2 * valPixels
    transY = numPixels * np.random.uniform() - numPixels/2
    transMat = np.float32([[1, 0, transX], [0, 1, transY]])
    image = cv2.warpAffine(image, transMat, (cols, rows))
    return image, steering



def straightdriving(data_df, hist_items=5):
    print('filtering straight line driving with %d frames consective' %
          hist_items)
    steering_history = deque([])
    drop_rows = []

    for idx, row in data_df.iterrows():
        
        steering = getattr(row, 'steering')

       
        steering_history.append(steering)
        if len(steering_history) > hist_items:
            steering_history.popleft()

       
        if steering_history.count(0.0) == hist_items:
            drop_rows.append(idx)

    
    return data_df.drop(data_df.index[drop_rows])



def cameraimage(row, log_path, cameras):
    steering = getattr(row, 'steering')
   
    camera = cameras[random.randint(0, len(cameras)-1)]
   
    steering += steering_adj[camera]

    image = loadImg(log_path, getattr(row, camera))
    image, steering = imagerotation(image, steering)
    image = brightness(image)

    return image, steering



def gen_train_data(log_path='./data', log_file='driving_log.csv', skiprows=1,
                   cameras=cameras, filter_straights=False,
                   crop_image=True, batch_size=128):

    # load the csv log file
    print("Cameras: ", cameras)
    print("Log path: ", log_path)
    print("Log file: ", log_file)

    column_names = ['center', 'left', 'right',
                    'steering', 'throttle', 'brake', 'speed']
    data_df = pd.read_csv(log_path+'/'+log_file,
                          names=column_names,skiprows=1)
    
    print(filter_straights)
    if filter_straights:
        data_df = filter_driving_straight(data_df)

    data_count = len(data_df)

    print("Log with %d rows." % (len(data_df)))
    i=0
    while (i<100):  # need to keep generating data

        # initialise data extract
        features = []
        labels = []
        
        # create a random batch to return
        while len(features) < batch_size:
            #print(features)
            row = data_df.iloc[np.random.randint(data_count-1)]
            i+=i
            image, steering = cameraimage(row, log_path, cameras)

            # flip 50% randomily that are not driving straight
            if random.random() >= .5 and abs(steering) > 0.1:
                image = cv2.flip(image, 1)
                steering = -steering

            if crop_image:
                image = crop_img(image)

            features.append(image)
            labels.append(steering)

        # yield the batch
        yield (np.array(features), np.array(labels))


# create a valdiation data generator for keras fit_model
def gen_val_data(log_path='/u200/Udacity/behavioral-cloning-project/data',
                 log_file='driving_log.csv', camera=camera_centre[0],
                 crop_image=True, skiprows=1,
                 batch_size=128):

    # load the csv log file
    print("Camera: ", camera)
    print("Log path: ", log_path)
    print("Log file: ", log_file)

    column_names = ['center', 'left', 'right',
                    'steering', 'throttle', 'brake', 'speed']
    data_df = pd.read_csv(log_path+'/'+log_file,
                          names=column_names, skiprows=skiprows)
    data_count = len(data_df)
    print("Log with %d rows."
          % (data_count))

    while data_count:  # need to keep generating data

        # initialise data extract
        features = []
        labels = []
 
        # create a random batch to return
        while len(features) < batch_size:
            #print(features)
            row = data_df.iloc[np.random.randint(data_count-1)]
            steering = getattr(row, 'steering')
           
            # adjust steering if not center
            steering += steering_adj[camera]

            image = loadImg(log_path, getattr(row, camera))

            if crop_image:
                image = crop_img(image)

            features.append(image)
            labels.append(steering)

        # yield the batch
        yield (np.array(features), np.array(labels))




def build_nvidia_model(img_height=66, img_width=200, img_channels=3,
                       dropout=.4):

    
    # normalisation layer
    inputShape = (img_height, img_width, img_channels)
    #model.add(Lambda(lambda x: x * 1./127.5 - 1,
    #                 input_shape=(img_shape),
     #                output_shape=(img_shape), name='Normalization'))
    
    
        # my architecture 
    model = Sequential()
    model.add(Lambda(lambda x: x/255.-0.5,input_shape=inputShape))
    model.add(Conv2D(3,1,1, activation = 'elu', name='Conv2D1')) # color space 

    model.add(Conv2D(16,8,8, activation = 'elu', name='Conv2D2'))
    #model.add(Conv2D(16,9,9, activation = 'elu', name='Conv2D2'))
    model.add(MaxPooling2D(pool_size=(2,2), name='MaxPoolC2'))
    model.add(Dropout(0.3, name='DropoutC2'))

    model.add(Conv2D(32,5,5, activation = 'elu', name='Conv2D3'))
    #model.add(Conv2D(32,7,7, activation = 'elu', name='Conv2D3'))
    model.add(MaxPooling2D(pool_size=(2,2), name='MaxPoolC3'))
    model.add(Dropout(0.3, name='DropoutC3'))

    model.add(Conv2D(32,3,3, activation = 'elu', name='Conv2D4'))
    #model.add(Conv2D(128,3,3, activation = 'elu', name='Conv2D5'))
    model.add(MaxPooling2D(pool_size=(2,2), name='MaxPoolC4'))
    model.add(Dropout(0.3, name='DropoutC4'))

    #model.add(Conv2D(64,3,3, activation = 'elu', name='Conv2D5'))
    #model.add(Conv2D(128,3,3, activation = 'elu', name='Conv2D6'))
    #model.add(MaxPooling2D(pool_size=(2,2), name='MaxPoolC6'))
    #model.add(Dropout(0.5, name='DropoutC6'))

    #model.add(Conv2D(256,2,2, activation = 'relu', name='Conv2D7'))
    #model.add(MaxPooling2D(pool_size=(2,2), name='MaxPoolC7'))
    #model.add(Dropout(0.5, name='DropoutC7'))

    # convolution to dense
    model.add(Flatten(name='Conv2Dense'))

    model.add(Dense(256,activation='elu', name='Dense1'))
    model.add(Dropout(0.5, name='DropoutD1'))

    model.add(Dense(128,activation='elu', name='Dense2'))
    model.add(Dropout(0.5, name='DropoutD2'))

    model.add(Dense(64,activation='elu', name='Dense3'))
    model.add(Dropout(0.5, name='DropoutD3'))

    model.add(Dense(8,activation='elu', name='Dense4'))
    model.add(Dropout(0.5, name='DropoutD4'))

    model.add(Dense(1,activation='elu', name='Output'))

 
   

    optimizer = Adam(lr=0.001)
    model.compile(optimizer=optimizer,
                  loss='mse')
    return model


def get_callbacks():
    # checkpoint = ModelCheckpoint(
    #     "checkpoints/model-{val_loss:.4f}.h5",
    #     monitor='val_loss', verbose=1, save_weights_only=True,
    #     save_best_only=True)

    # tensorboard = TensorBoard(log_dir='./logs', histogram_freq=0,
    #                           write_graph=True, write_images=False)

    # return [checkpoint, tensorboard]

    earlystopping = EarlyStopping(monitor='val_loss', min_delta=0,
                                  patience=1, verbose=1, mode='auto')
    # return [earlystopping, checkpoint]
    return [earlystopping]


def main():

    
    model = build_nvidia_model()
    
    print(model.summary())

    #plot(model, to_file='model.png', show_shapes=True)

    model.fit_generator(
        gen_train_data(log_path='./data',
                       cameras=cameras,
                       #    cameras=camera_centre,
                       crop_image=True,
                       batch_size=28
                       ),
        samples_per_epoch=20000,
        nb_epoch=3,
        callbacks=get_callbacks(),
        validation_data=gen_val_data(log_path='./data',
                                     crop_image=True,
                                     batch_size=28),
        nb_val_samples=5000)

    # save weights and model
    model.save('model.h5')
    with open('model.json', 'w') as modelfile:
        json.dump(model.to_json(), modelfile)



In [5]:
# parses flags and calls the `main` function above
if __name__ == '__main__':
    main()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:211: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(3, (1, 1), activation="elu", name="Conv2D1")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:213: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (8, 8), activation="elu", name="Conv2D2")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:218: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (5, 5), activation="elu", name="Conv2D3")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:223: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="elu", name="Conv2D4")`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_1 (Lambda)            (None, 66, 200, 3)        0         
_________________________________________________________________
Conv2D1 (Conv2D)             (None, 66, 200, 3)        12        
_________________________________________________________________
Conv2D2 (Conv2D)             (None, 59, 193, 16)       3088      
_________________________________________________________________
MaxPoolC2 (MaxPooling2D)     (None, 29, 96, 16)        0         
_________________________________________________________________
DropoutC2 (Dropout)          (None, 29, 96, 16)        0         
_________________________________________________________________
Conv2D3 (Conv2D)             (None, 25, 92, 32)        12832     
_________________________________________________________________
MaxPoolC3 (MaxPooling2D)     (None, 12, 46, 32)        0         
__________

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:302: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:302: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., callbacks=[<keras.ca..., validation_data=<generator..., steps_per_epoch=20000, epochs=3, validation_steps=5000)`


Epoch 1/3
Camera: Cameras:  center 
['left', 'center', 'right']Log path: 
 Log path: ./data 
./dataLog file: 
 Log file: driving_log.csv 
driving_log.csv
Log with 8036 rows.
False
Log with 8036 rows.
19999/20000 [============================>.] - ETA: 0s - loss: 0.0396

NameError: ignored

In [0]:
model = build_nvidia_model(dropout=.4)
print(model.summary())
#plot(model, to_file='model.png', show_shapes=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:249: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(24, (5, 5), activation="elu", strides=(2, 2), padding="valid")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:249: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(36, (5, 5), activation="elu", strides=(2, 2), padding="valid")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:249: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (5, 5), activation="elu", strides=(2, 2), padding="valid")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:249: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="elu", strides=(1, 1), padding="valid")`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Normalization (Lambda)       (None, 66, 200, 3)        0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 31, 98, 24)        1824      
_________________________________________________________________
dropout_25 (Dropout)         (None, 31, 98, 24)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 14, 47, 36)        21636     
_________________________________________________________________
dropout_26 (Dropout)         (None, 14, 47, 36)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 5, 22, 48)         43248     
_________________________________________________________________
dropout_27 (Dropout)         (None, 5, 22, 48)         0         
__________

NameError: ignored

In [0]:
model.fit_generator(
        gen_train_data(log_path='./data',
                       cameras=cameras,
                       #    cameras=camera_centre,
                       crop_image=True,
                       batch_size=36
                       ),
        samples_per_epoch=30000,
        nb_epoch=5,
        callbacks=get_callbacks(),
        validation_data=gen_val_data(log_path='./data',
                                     crop_image=True,
                                     batch_size=36),
        nb_val_samples=5000)

Epoch 1/5
Cameras:  ['left', 'center', 'right']
Log path:  ./data
Log file:  driving_log.csv
Camera:  center
Log path:  ./data
Log file:  driving_log.csv
FalseLog with 8036 rows.

Log with 8036 rows.
    1/30000 [..............................] - ETA: 1:30:46 - loss: 0.0163

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., callbacks=[<keras.ca..., validation_data=<generator..., steps_per_epoch=30000, epochs=5, validation_steps=5000)`
  


30000/30000 [==============================] - 2164s 72ms/step - loss: 0.0238 - val_loss: 0.0157
Epoch 2/5
30000/30000 [==============================] - 2157s 72ms/step - loss: 0.0249 - val_loss: 0.0130
Epoch 3/5
    8/30000 [..............................] - ETA: 23:07 - loss: 0.0233Buffered data was truncated after reaching the output size limit.